In [1]:
import pandas as pd

In [3]:
%run ../../functions.ipynb

### Import data & model

In [8]:
#get sample dataset
sub = pd.read_csv('../../../data/MSampleSubmissionStage1_2020.csv')
sub = process_sample_sub(sub)

In [9]:
#processed data
all_games = pd.read_csv('../output/all_games.csv')


In [10]:
#model
import pickle

with open("../model/logreg.pkl", "rb") as file: 
    logreg = pickle.load(file)
    
with open("../model/rf.pkl", "rb") as file: 
    rf = pickle.load(file)

with open("../model/oe_model.pkl", "rb") as file: 
    oe_model = pickle.load(file)
    
with open("../model/de_model.pkl", "rb") as file: 
    de_model = pickle.load(file)


### Get data for prediction

In [11]:
#get df for end of the season for Team1
season_end = all_games.groupby(['Team1', 'Season'])['DayNum'].max().reset_index()
season_end_df = all_games.merge(season_end, on = ['Team1', 'Season', 'DayNum'])


In [12]:
#join in team1 data
features = [col for col in all_games.columns if col.startswith('Team1_')]
join_key = ['Season', 'Team1']
sub = sub.merge(season_end_df[features + join_key], on = join_key)

In [13]:
season_end_df = season_end_df[[col for col in season_end_df.columns if not col.startswith('Team2_')]]


In [14]:
#get rid of team2
season_end_df = season_end_df[[col for col in season_end_df.columns if not col.startswith('Team2')]]
#map team1 to team2
season_end_df.rename(columns = {col: col.replace('Team1', 'Team2') for col in season_end_df.columns}, inplace = True)
#join in team2
features = [col for col in season_end_df.columns if col.startswith('Team2_')]
join_key = ['Season', 'Team2']
sub = sub.merge(season_end_df[features + join_key], on = join_key)


In [15]:
#get oe and de features

reg_feature_list = ['Team1_avg_oe', 'Team1_avg_de','Team1_avg_opp_avg_oe','Team1_avg_opp_avg_de',
                           'Team1_avg_opp_avg_opp_avg_oe', 'Team1_avg_opp_avg_opp_avg_de',
                            'Team1_avg_opp_avg_opp_avg_opp_avg_oe', 'Team1_avg_opp_avg_opp_avg_opp_avg_de',
                           'Team1_avg_opp_avg_opp_avg_opp_avg_opp_avg_oe',
                               'Team1_avg_opp_avg_opp_avg_opp_avg_opp_avg_de',
                    
                   'Team2_avg_oe', 'Team2_avg_de','Team2_avg_opp_avg_oe','Team2_avg_opp_avg_de',
                           'Team2_avg_opp_avg_opp_avg_oe', 'Team2_avg_opp_avg_opp_avg_de',
                            'Team2_avg_opp_avg_opp_avg_opp_avg_oe', 'Team2_avg_opp_avg_opp_avg_opp_avg_de',
                           'Team2_avg_opp_avg_opp_avg_opp_avg_opp_avg_oe',
                               'Team2_avg_opp_avg_opp_avg_opp_avg_opp_avg_de']


sub['t1_pred_oe'] = oe_model.predict(sub[reg_feature_list].values)
sub['t1_pred_de'] = de_model.predict(sub[reg_feature_list].values)

### Prediction

In [18]:
feature_list = ['t1_pred_oe', 't1_pred_de']
sub['Pred'] = logreg.predict_proba(sub[feature_list])[:,1]

In [19]:
sub[['ID', 'Pred']].to_csv('submission.csv', index = False)